In [2]:
import os, pathlib, datetime as dt
import pandas as pd
from dotenv import load_dotenv
import numpy as np

load_dotenv()
RAW = pathlib.Path(os.getenv('DATA_DIR_RAW', 'data/raw'))
PROC = pathlib.Path(os.getenv('DATA_DIR_PROCESSED', 'data/processed'))
RAW.mkdir(parents=True, exist_ok=True)
PROC.mkdir(parents=True, exist_ok=True)
print('RAW ->', RAW.resolve())
print('PROC ->', PROC.resolve())

dates = pd.date_range('2024-01-01', periods=20, freq='D')
df = pd.DataFrame({'date': dates, 'ticker': ['AAPL']*20, 'price': 150 + np.random.randn(20).cumsum()})
df.head()

RAW -> C:\Users\kvssr\bootcamp_rohit_kakumani\homework\homework5\data\raw
PROC -> C:\Users\kvssr\bootcamp_rohit_kakumani\homework\homework5\data\processed


,date,ticker,price
0,2024-01-01,AAPL,149.612695
1,2024-01-02,AAPL,150.348904
2,2024-01-03,AAPL,149.986891
3,2024-01-04,AAPL,149.922273
4,2024-01-05,AAPL,148.877593


In [3]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install fastparquet

Note: you may need to restart the kernel to use updated packages.


In [5]:

# Save to CSV (raw)
csv_path = os.path.join(RAW, "aapl_prices.csv")
df.to_csv(csv_path, index=False)


# Save to Parquet (processed)
parquet_path = os.path.join(PROC, "aapl_prices.parquet")
try:
    df.to_parquet(parquet_path)
except Exception as e:
    print('Parquet engine not available. Install pyarrow or fastparquet to complete this step.')

print(f"✅ Saved CSV to {csv_path}")
print(f"✅ Saved Parquet to {parquet_path}")

✅ Saved CSV to ..\data\raw\aapl_prices.csv
✅ Saved Parquet to ..\data\processed\aapl_prices.parquet


In [7]:
df_csv = pd.read_csv(csv_path, parse_dates=["date"])
df_parquet = pd.read_parquet(parquet_path)


def validate_dataframes(df1, df2, critical_cols):
    results = {}

    # Check shapes
    results["shape_match"] = df1.shape == df2.shape

    # Check critical columns exist
    results["columns_exist"] = all(col in df2.columns for col in critical_cols)

    # Check dtypes
    dtype_check = {}
    for col in critical_cols:
        dtype_check[col] = (df1[col].dtype == df2[col].dtype)
    results["dtypes_match"] = dtype_check

    # Check NA counts
    na_check = {}
    for col in critical_cols:
        na_check[col] = (df1[col].isna().sum(), df2[col].isna().sum())
    results["na_counts"] = na_check

    return results

# Run validation
validation_results = validate_dataframes(df_csv, df_parquet, critical_cols=["date", "ticker", "price"])
print("✅ Validation Results:")
print(validation_results)

✅ Validation Results:
{'shape_match': True, 'columns_exist': True, 'dtypes_match': {'date': True, 'ticker': True, 'price': True}, 'na_counts': {'date': (np.int64(0), np.int64(0)), 'ticker': (np.int64(0), np.int64(0)), 'price': (np.int64(0), np.int64(0))}}
